# Scraping many pages + Using Selenium

## The pages we'll be looking at

If I wanted to read specific information about a specfic mine, it takes a few steps. **Do these steps with your browser before you try any programming.**

1. Visit the [Mine Data Retrieval System](https://arlweb.msha.gov/drs/drshome.htm)
2. Scroll down to **Mine Identification Number (ID) Search**
3. Type in a mine ID number, such as `3503598`, click **Search**
4. I'm on a page! It lists the MINE NAME and MINE OWNER.

After searching for and finding a mine, I can use this page to **find reports about this mine**. Some of the reports are on accidents, violations, inspections, health samples and more. To get those reports:

1. Search for a mine (if you haven't already)
2. Scroll down and change **Beginning Date** to `1/1/1995` (violation reports begin in 1995, accidents begin in 1983)
3. Select the report type of `Violations`
4. Click **Get Report**
5. I'm on a page! It lists ALL OF THE MINE'S VIOLATIONS.

By changing the report type you're searching for you can find all sorts of different data.

# Researching mine information

## Preparation 

### When you search for information on a specific mine, what URL should Selenium visit first?

- *TIP: the answer is NOT `https://arlweb.msha.gov/drs/ASP/BasicMineInfonew.asp`*

In [60]:
# https://arlweb.msha.gov/drs/drshome.htm

### How can you identify the text field we're going to type the Mine ID into?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

In [61]:
# //*[@id="inputdrs"]

### How can you identify the search button we're going to click, or the form we're going to submit?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [62]:
# The button's xpath is:
# //*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input

### Use Selenium to search using the mine ID `3901432`. Get me the operator's name by scraping.

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

In [63]:
from selenium import webdriver

In [64]:
driver = webdriver.Chrome()

In [65]:
driver.get('https://arlweb.msha.gov/drs/drshome.htm')

In [66]:
input_field = driver.find_element_by_name("MineId")
input_field

<selenium.webdriver.remote.webelement.WebElement (session="df3396580b42106e6143c8a8c7905c34", element="0.8221011958838298-1")>

In [67]:
input_field.send_keys("3901432")

In [68]:
search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

In [69]:
name_tag = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[1]/tbody/tr[3]/td[2]/font/b')

In [70]:
name_tag.text

'Krueger Brothers Gravel & Dirt'

# Using .apply to find data about SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

In [71]:
import pandas as pd

df = pd.read_csv("mines-subset.csv", dtype={'id':'str'})
df.head()

,id
0,4104757
1,0801306
2,3609931


In [72]:
!cat mines-subset.csv

id
4104757
0801306
3609931

### Open up `mines-subset.csv` in a text editor, then look at your dataframe. Is something different about them?

### Scrape the operator's name for each of those mines and print it

- *TIP: use .apply and a function*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook *

In [73]:
driver = webdriver.Chrome()

In [74]:
def scrape_name(row):
    driver.get("https://arlweb.msha.gov/drs/drshome.htm")
    input_field = driver.find_element_by_name("MineId")
    input_field.send_keys(row['id'])
    search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
    search_button.click()
    name_tag = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[1]/tbody/tr[3]/td[2]/font/b')
    return name_tag.text
    
driver = webdriver.Chrome()
df.apply(scrape_name, axis=1)

0                  Dirt Works
1    Holley Dirt Company, Inc
2              M.R. Dirt Inc.
dtype: object

### Scrape the operator's name and save it into a new column

- *TIP: Use .apply and a function*
- *TIP: Remember to use `return`*

In [75]:
df['name'] = df.apply(scrape_name, axis=1)

In [76]:
df.head()

,id,name
0,4104757,Dirt Works
1,0801306,"Holley Dirt Company, Inc"
2,3609931,M.R. Dirt Inc.


# Researching mine violations

Read the very top again to remember how to find mine violations

### When you search for a mine's violations, what URL is Selenium going to start on?

- *TIP: `requests` can send form data to load in the middle of a bunch of steps, but Selenium has to start at the beginning

In [77]:
# https://arlweb.msha.gov/drs/drshome.htm

### When you're searching for violations from the Mine Information page, how are you going to identify the "Beginning Date" field?

In [78]:
#//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]

### When you're searching for violations from the Mine Information page, how are you going to identify the "Violations" button?

In [79]:
#//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input

### When you're searching for violations from the Mine Information page, how are you going to identify the form or the button to click to get a list of the violations?

In [80]:
#//*[@id="content"]/form[1]/table[3]/tbody/tr[3]/td[2]/input

### Using the mine ID `3901432`, scrape all of their violations since 1/1/1995

**Save this into a CSV called `3901432-violations.csv`.** This CSV must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

**Tips:**

- *TIP: It's probably worth it to print them all first, then save them to a CSV once you know it's all working.*
- *TIP: You'll use the parent pattern - get the ROWS first (tr), then loop through and get the TABLE CELLS (td)*

In [81]:
# get to the main line page
driver.get("https://arlweb.msha.gov/drs/drshome.htm")
input_field = driver.find_element_by_name("MineId")
input_field.send_keys('3901432')
search_button = driver.find_element_by_xpath('//*[@id="content"]/table[3]/tbody/tr[3]/td[2]/input')
search_button.click()

In [82]:
date_field = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[2]/tbody/tr[2]/td/font/input[1]')
date_field.send_keys("1/1/1995")

In [83]:
violation_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
violation_button.click()

In [84]:
reports_button = driver.find_element_by_xpath('//*[@id="content"]/form[1]/table[3]/tbody/tr[2]/td[2]/table/tbody/tr[1]/td/input')
reports_button.click()

In [85]:
#name
#Class
#ID
#CSS selector (ASK ME WHAT THIS IS if you don't know)
#XPath (ASK ME WHAT THIS IS because you definitely don't know)
#Link text
#Partial link text

In [86]:
from bs4 import BeautifulSoup

In [87]:
doc = BeautifulSoup(driver.page_source, 'html.parser')

In [90]:
violation_rows = doc.find_all('tr', class_='drsviols')
len(violation_rows)

0

In [89]:
for violation_row in violation_rows: 
    print("I have a violation")
    cells = violation_row.find_all('td')
    print("Citation no", cells[2].text)
    print("Case no", cells[3].text)
    print("Standard violated", cells[10].find('a').text)
    print("Link", cells[10].find_all('a')['href'])
    print("Proposed Penalty", cells [11].text)
    print("Amount paid to date", cells[14].text)
         
       
    print("I found", len(cells), "td elements")
    print("---------------")

SyntaxError: invalid syntax (<ipython-input-89-8d977a4d893d>, line 4)

In [ ]:
#to save for csv:
#make an empty list
#every time through the loop, create a dictionara of your data
#at the end of the run through the loop
#when the entire loopi is over, convert in the lisr to dataframe
#Save that dataframe
violations = []
for violation_row in violation_rows: 
    violation{}
    print("I have a violation")
    cells = violation_row.find_all('td')
    violation['citation'] = cells[2].text.strip()
    violation['case no'] = cells[3].text.strip()
    violation['standard'] = cells[10].find('a').text.strip()
    violation['link'] = cells[10].find('a')['href']
    violation['proposed'] = cells[11].text.strip()
    violation['paid'] = cells[14].text.strip()
    violations.append(violation)

In [ ]:
violations

In [ ]:
import pandas as pd

df = pd.DataFrame()
df.head()

In [ ]:
df.to_csv("3901432-violations.csv"), index=False)

# Using .apply to save mine data for SEVERAL mines

The file `mines-subset.csv` has a list of mine IDs. We're going to scrape the operator's name for each of those mines.

### Open up `mines-subset.csv` and save it into a dataframe

### Scrape the violations for each mine

**Save each mine's violations into separate CSV files.** Each CSV file must include the following fields:

- Citation number
- Case number
- Standard violated
- Link to standard
- Proposed penalty
- Amount paid to date

Make sure you are saving them into **separate files.** It might be nice to name them after the mine id.

- *TIP: Use .apply for this*
- *TIP: Print out the ID before you start scraping. That way you can take that ID and search manually to see if there is anything weird about the results.*
- *TIP: If you need help with .apply, look at the "Using apply in pandas" notebook 
- *TIP: It's probably worth it to print the fields first, then save them to a CSV once you know it's all working.*